# 小百合JobExpress板块信息
+ 储存到阿里云mysql中

In [1]:
from requests_html import HTMLSession
import regex as re
import string
import time
from mylog import MyLog as mylog
import pymysql
import pickle
from tqdm import tqdm

In [2]:
SUCCESSES = 0
ERRORS = 0

In [3]:
logger = mylog()
htmlsession = HTMLSession()
headers = {
    'user-agent':
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:54.0) Gecko/20100101 Firefox/54.0',
    'accept':
    'text/html, application/xhtml+xml, application/xml; q=0.9, image/webp, */*; q=0.8',
    'referer':
    'http://bbs.nju.edu.cn/g/JobExpress'
}

In [4]:
start_url = 'http://bbs.nju.edu.cn/bbsdoc?board=JobExpress'
pattern_index = re.compile('bbscon\?board=JobExpress&file=M\.\d+\.A&num=(\d+)')

In [5]:
keep_going = True
count_articles = 999999
url_articles = []

In [6]:
db = pymysql.connect("39.108.157.74","root","00genius00","JobExpress" )
cursor = db.cursor()

In [7]:
cursor.execute("SELECT MAX(id) FROM links")

1

In [8]:
count_crawled = cursor.fetchone()[0]

In [9]:
def update_url_articles(res):
    global count_articles
    global keep_going
    global pattern_index
    for link in res.html.links:
        if link.startswith('bbscon?board=JobExpress&file=M.'):
            index = int(pattern_index.match(link).group(1))
            if index > count_crawled:
                count_articles = index if count_articles > index else count_articles
                url_articles.append((str(index), 'http://bbs.nju.edu.cn/'+link))
            else:
                keep_going = False

In [10]:
res = htmlsession.get(start_url, headers=headers)
update_url_articles(res)

In [11]:
while keep_going:
    count_articles = count_articles-20 if count_articles>=20 else 0
    target_url = 'http://bbs.nju.edu.cn/bbsdoc?board=JobExpress&start=%s&type=doc' %str(count_articles)
    res = htmlsession.get(target_url, headers=headers)
    update_url_articles(res)
    time.sleep(0.5)

In [21]:
def insert(cursor, db, index, url):
    global SUCCESSES, ERRORS
    sql_query = "INSERT IGNORE INTO links VALUES ('%s', '%s')" %(str(index), url)
    try:
        cursor.execute(sql_query)
        db.commit()
        logger.info('成功: 插入id为: %s 的招聘信息' %str(index))
        SUCCESSES += 1
    except:
        db.rollback()
        logger.error('失败: 插入id为: %s 的招聘信息' %str(index))
        ERRORS += 1

In [22]:
for item in tqdm(url_articles):
    insert(cursor, db, item[0], item[1])

  0%|                                                                           | 0/30 [00:00<?, ?it/s]2018-08-28 01:28:57,859 INFO     ydzhang    成功: 插入id为: 12020 的招聘信息

2018-08-28 01:28:57,935 INFO     ydzhang    成功: 插入id为: 12026 的招聘信息

  7%|████▍                                                              | 2/30 [00:00<00:02, 13.26it/s]2018-08-28 01:28:58,013 INFO     ydzhang    成功: 插入id为: 12013 的招聘信息

2018-08-28 01:28:58,087 INFO     ydzhang    成功: 插入id为: 12015 的招聘信息

 13%|████████▉                                                          | 4/30 [00:00<00:01, 13.21it/s]2018-08-28 01:28:58,162 INFO     ydzhang    成功: 插入id为: 12023 的招聘信息

2018-08-28 01:28:58,236 INFO     ydzhang    成功: 插入id为: 12030 的招聘信息

 20%|█████████████▍                                                     | 6/30 [00:00<00:01, 13.26it/s]2018-08-28 01:28:58,310 INFO     ydzhang    成功: 插入id为: 12019 的招聘信息

2018-08-28 01:28:58,384 INFO     ydzhang    成功: 插入id为: 12017 的招聘信息

 27%|█████████████████▊                     

In [23]:
print(SUCCESSES, ERRORS)

30 30
